# 검색 알고리즘 실험
> 특정 문서를 입력받아 코사인 유사도를 기반으로 유사한 문서를 반환하는 실험을 진행해보자.

1. TF-IDF + NMF
1. Word2vec
1. Fasttext

## Import

In [11]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.preprocessing import Normalizer
from konlpy.tag import Mecab
from gensim.models import Word2Vec
from gensim.models import FastText

## Data
>sourced by momtalk(https://www.momtalk.kr/talk/list/all)

1. `df1`: 육아정보 포스팅
1. `df2`: 유아용품 포스팅
1. `df` : 위의 두 데이터를 합친 것

In [2]:
df1 = pd.read_csv("data/maeili.csv", encoding='utf-8')
df2 = pd.read_csv("data/baby.csv", encoding='utf-8')
df3 = pd.read_csv("data/toys.csv", encoding='utf-8')
df4 = pd.read_csv("data/seoulchildcare.csv", encoding='utf-8')
df5 = pd.read_csv("data/babynews_DLC.csv", encoding='utf-8')
df6 = pd.read_csv("data/ildongfoodis.csv", encoding='utf-8')
df7 = pd.read_csv("data/momsmagazine.csv", encoding='utf-8')
df8 = pd.read_csv("data/ange.csv", encoding='utf-8')

df = pd.concat([df1.text, df2.text, df3.text, df4.text, 
                df5.text, df6.text, df7.text, df8.text])
df = df.drop(index=[952, 955, 956, 960, 962, 964, 1165, 1240, 1322])
df = df.dropna()
df = pd.DataFrame(df).reset_index(drop=True)
df.tail()

,text
17301,아이는 현재 신체와 정신이 모두 발달 중에 있다 신체 발달은 근육의 발달을 정신 발...
17302,세 살 적 버릇이 여든까지 간다는 말이 있다 밥상머리 예절도 예외는 아니다 아이에게...
17303,아이의 키가 자라고 몸무게가 늘어나면서 아이 몸을 구성하고 있는 장기들도 커지고 무...
17304,눈치는 직관적이고 비언어적인 방법으로 의사소통을 하고 문제를 해결하는 놀라운 능력이...
17305,소리를 들으면 사람은 그 소리를 그대로 흉내 낼 수 있다 이것이 청력의 힘이다 말소...


## TF-IDF

In [3]:
def tokenize(text):
    mecab = Mecab()
    tokens = mecab.pos(text)
    key_tokens = [token[0] for token in tokens 
                 if ('NNG' in token or 'NNP' in token or 'VA' in token or 'VV' in token)]
    key_tokens = ' '.join(key_tokens)
    return key_tokens

    
df['token'] = df.text.apply(tokenize)

In [4]:
df.tail()

,text,token
17301,아이는 현재 신체와 정신이 모두 발달 중에 있다 신체 발달은 근육의 발달을 정신 발...,아이 신체 정신 발달 있 신체 발달 근육 발달 정신 발달 지적 호기심 발달 포함 공...
17302,세 살 적 버릇이 여든까지 간다는 말이 있다 밥상머리 예절도 예외는 아니다 아이에게...,버릇 말 있 밥상머리 예절 예외 아이 식습관 가족 밥상 앞 앉 우선 가족 자신 자리...
17303,아이의 키가 자라고 몸무게가 늘어나면서 아이 몸을 구성하고 있는 장기들도 커지고 무...,아이 키 자라 몸무게 늘어나 아이 몸 구성 장기 무게 성장 발달 신체 부분 일정 순...
17304,눈치는 직관적이고 비언어적인 방법으로 의사소통을 하고 문제를 해결하는 놀라운 능력이...,눈치 직관 언어 방법 의사소통 하 문제 해결 능력 사람 관계 맺 사람 생각 감정 이...
17305,소리를 들으면 사람은 그 소리를 그대로 흉내 낼 수 있다 이것이 청력의 힘이다 말소...,소리 사람 소리 흉내 있 청력 힘 말 소리 아이 듣 소리 모방 소리 소리 감정 느낌...


In [5]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(df['token'])

print(vectors.shape)

(17306, 52618)


## NMF(Non-negative Matrix Factorization)
- sparse coding처럼 dictionary와 encoded vector 학습
- parameter가 0보다 큰 특성으로 feature matrix가 원본 데이터들과 어떠한 관계를 가지는지 해석이 가능
- 연산에 약간의 시간 소요

In [6]:
%%time
vector_array = vectors.toarray()
nmf = NMF(n_components=40)
nmf.fit(vector_array)
features = nmf.transform(vector_array)

CPU times: user 15min 9s, sys: 8.1 s, total: 15min 17s
Wall time: 2min 1s


## Normalize extracted feature set
- 0과 1사이의 값으로 정규화하여 feature vector의 길이를 1로 고정

In [7]:
normalizer = Normalizer()
norm_features=normalizer.fit_transform(features)

print(norm_features[0:2])

[[0.71261336 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.00389628 0.         0.         0.         0.592812
  0.20593723 0.14062457 0.         0.         0.         0.18695898
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.18449045 0.09780161 0.
  0.         0.         0.         0.        ]
 [0.50994692 0.         0.         0.10420228 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.81103146 0.         0.         0.         0.01620445 0.16409557
  0.         0.         0.14205294 0.         0.         0.06022933
  0.0918441  0.06572892 0.         0.08569398 0.         0.0151111
  0.         0.         0.         0.        ]]


In [8]:
df_features = pd.DataFrame(norm_features,index=df.index.tolist())
df_features.tail()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
17301,0.469231,0.0,0.000000,0.003950,0.023671,0.000000,0.0,0.000000,0.000000,0.067784,...,0.088754,0.000000,0.03851,0.000000,0.000000,0.000000,0.019101,0.0,0.0,0.017121
17302,0.651384,0.0,0.000000,0.034809,0.000000,0.000000,0.0,0.000000,0.000000,0.448324,...,0.016399,0.300472,0.00000,0.131928,0.000000,0.000000,0.118490,0.0,0.0,0.000000
17303,0.150421,0.0,0.023623,0.024743,0.000000,0.000000,0.0,0.010280,0.000000,0.000000,...,0.183334,0.000000,0.00000,0.005918,0.000000,0.053625,0.000000,0.0,0.0,0.000000
17304,0.428456,0.0,0.000000,0.188804,0.003426,0.006913,0.0,0.028218,0.013103,0.000000,...,0.000000,0.027621,0.00000,0.000000,0.039106,0.000000,0.000000,0.0,0.0,0.000000
17305,0.026630,0.0,0.000000,0.026475,0.000000,0.121803,0.0,0.000000,0.000000,0.000000,...,0.065552,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000


`df`의 맨 첫번째 텍스트를 정규화 한 40차원 벡터의 값

In [9]:
print(df.loc[0].text)
article = df_features.loc[0]
len(article)

6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요  매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다 


40

In [10]:
similarities=df_features.dot(article)
top=similarities.nlargest(10)

texts = df.loc[top.index]['text'].tolist()
i = 0
for text in texts:
    df_idx = top.index[i]
    print('TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
    #print(text+'\n')
    i = i+1

TITLE :0 Similarities:1.0
TITLE :1788 Similarities:0.9408881812088815
TITLE :869 Similarities:0.9378332493233649
TITLE :14011 Similarities:0.9231710618772622
TITLE :1801 Similarities:0.9124326341052716
TITLE :260 Similarities:0.9096111881462734
TITLE :11623 Similarities:0.907023712152378
TITLE :5811 Similarities:0.9062118106424178
TITLE :8867 Similarities:0.8990816489524339
TITLE :9830 Similarities:0.8970800266271007


In [11]:
i = 0
for text in texts:
    df_idx = top.index[i]
    print('TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
    print(text+'\n')
    i = i+1

TITLE :0 Similarities:1.0
6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요  매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다 

TITLE :1788 Similarities:0.9408881812088815
 아이가 갑자기 학원을 안 가겠대요 이미지 아이아트그만둬도 될까 아이가 갑자기 학원을 안 가겠대요 아이가 너무 원해서 발레학원을 다니기 

---

## Word2Vec
- 위의 TF-IDF 임베딩이 아닌 Word2Vec 임베딩으로 실험해보자.
- 음수의 값이 포함되서는 안되는 NMF는 활용할 수 없다. 어차피 sparse하지 않으면서 TF-IDF에 비해 훨씬 저차원인 성격을 지닌 word2vec 임베딩은 바로 유사도를 계산할 수 있어 더욱 편리하게 활용할 수 있을 것이라 판단된다.

In [105]:
w2v_model = Word2Vec.load('data/tokmom_word2vec_all')
df = pd.read_csv('data/text_for_word2vec.csv')
df = df.drop(index=df[df['text'] == ""].index)
df = df.dropna()
df = df.reset_index(drop=True)

In [106]:
def tokenize(text):
    tokenizer = Mecab()
    text = tokenizer.morphs(text)
    return text

In [107]:
df['tokenized_text'] = df['text'].apply(tokenize)

In [108]:
def make_to_vector(tokens):
    mean_of_vectors = []
    except_tokens = []
    for token in tokens:
        try:
            vector = w2v_model.wv.get_vector(token)
            mean_of_vectors.append(vector)
        except KeyError:
#             print("{} is not exist in the vocab".format(noun))
            except_tokens.append(token)
            continue
    try:
        mean_of_vectors = sum(mean_of_vectors) / len(mean_of_vectors)
        return mean_of_vectors
    except ZeroDivisionError:
        return np.nan

In [109]:
df['doc_vector'] = df['tokenized_text'].apply(make_to_vector)
df = df.dropna()
df = df.reset_index(drop=True)

In [110]:
vectors = np.zeros(128,)
for vector in df.doc_vector:
    vectors = np.vstack([vectors, vector])
vectors = vectors[1:, :]
vectors

array([[-0.00423659,  0.17780623,  0.20986421, ..., -0.09617404,
         0.2035376 ,  0.10911077],
       [-0.02205996,  0.17980617,  0.16847241, ..., -0.1201755 ,
         0.18646029,  0.11190627],
       [ 0.01685894,  0.18243246,  0.1859199 , ..., -0.1138395 ,
         0.18536329,  0.08026116],
       ...,
       [-0.05851585,  0.10892133,  0.23614064, ..., -0.08655355,
         0.1845157 ,  0.0366469 ],
       [ 0.03773687,  0.15562312,  0.19995259, ..., -0.09915192,
         0.21535033,  0.11777136],
       [ 0.05184415,  0.18287562,  0.21582051, ..., -0.09163125,
         0.23449181,  0.10614546]])

>ndarray 형식으로 예쁘게 만들어야 되는데 이런 식으로 밖에 못만드는게 내 실력 미달인듯... ㅠ

## Normalize extracted feature set
- 유사도 계산의 편의를 위해 0과 1사이의 값으로 정규화

In [111]:
from sklearn.preprocessing import Normalizer

normalizer = Normalizer()
norm_features=normalizer.fit_transform(vectors)
df_features = pd.DataFrame(norm_features,index=df.index.tolist())
df_features.tail()

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
19499,0.100223,-0.015523,0.125701,0.002736,0.059570,0.002465,0.083391,-0.009936,-0.154709,-0.087337,...,0.096420,-0.096232,0.033048,-0.001364,0.002103,-0.025324,0.006432,-0.128888,0.019098,0.054237
19500,0.039731,0.070507,0.139957,-0.108033,0.128847,0.151288,0.235687,0.082549,0.007348,-0.086715,...,0.137869,-0.073415,0.059888,-0.001680,-0.041405,0.054525,0.078836,-0.081503,0.169226,0.059253
19501,-0.036917,0.068717,0.148978,-0.012886,0.180859,0.128598,0.259999,0.023207,0.057743,-0.101745,...,0.139774,-0.063850,-0.009105,-0.066711,-0.019973,-0.037450,0.068126,-0.054605,0.116408,0.023120
19502,0.024387,0.100570,0.129218,-0.117767,0.168201,0.103840,0.169654,0.068522,-0.006473,-0.083712,...,0.139642,-0.099281,0.035922,-0.031027,-0.065497,0.019676,0.068444,-0.064076,0.139168,0.076109
19503,0.033040,0.116546,0.137542,-0.118049,0.148602,0.113015,0.199667,0.069674,0.019346,-0.087894,...,0.122310,-0.081414,0.061879,-0.019699,-0.061451,0.009648,0.083389,-0.058397,0.149441,0.067646


In [112]:
article = df_features.loc[0]
similarities=df_features.dot(article) # cosine similarity
top=similarities.nlargest(5)

texts = df.loc[top.index]['text'].tolist()

i = 0
print("<비교할 문서>")
for text in texts:
    df_idx = top.index[i]
    print('TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
    print(text+'\n')
    i = i+1

<비교할 문서>
TITLE :0 Similarities:1.0000000000000004
6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요 매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다

TITLE :17364 Similarities:0.9901152437787631
어린이교사학부모의 어려움 감싸는 따뜻한 울타리가 되고 싶어원 운영에서 교사가 가장 중요합니다 교

### 가상의 키워드를 입력하여 유사도 계산해보기
- 이번에는 챗봇처럼 짧은 문장(아마 키워드 위주)을 입력받아 유사도 계산을 해보자.
- word2vec vocabulary에 없는 단어를 입력받을 경우 에러가 발생하는 문제가 있다.

In [115]:
tokenizer = Mecab()

while True:
    input_text = input("Input text:")
    if input_text == "exit":
        print("Bye")
        break
        
    tokens = tokenizer.morphs(input_text)
    input_vector = []
    
    for token in tokens:
        try:
            word_vector = w2v_model.wv.get_vector(token)
            input_vector.append(word_vector)
        except:
            print("'{}'은(는) 제가 아직 잘 이해하지 못하겠어요ㅠㅠ ".format(token))
    
    input_vector = sum(input_vector) / len(input_vector)
    input_vector = input_vector.reshape(1, -1)
    input_vector = normalizer.fit_transform(input_vector)
    input_vector = sum(input_vector) / len(input_vector)
    input_vector = input_vector.reshape(-1,)

    similarities=df_features.dot(input_vector)
    top=similarities.nlargest(5)

    texts = df.loc[top.index]['text'].tolist()

    i = 0
    for text in texts:
        df_idx = top.index[i]
        print('\n'+'TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
        print(text+'\n')
        i = i+1
    

Input text:중고 육아용품 어디서 구매하는게 안전할까요?
'?'은(는) 제가 아직 잘 이해하지 못하겠어요ㅠㅠ 

TITLE :1609 Similarities:0.9115267889692947
해외직구사이트 육아용품 여기서 마련하세요 이미지 출처 아이아트 육아는 글로벌 장비빨 해외직구사이트 육아용품 여기서 마련하세요 안 해본 사람은 있어도 한 번만 해본 사람은 없다는 육아용품 해외직구 임신 출산 용품 부터 아기 옷 기저귀 장난감까지 국내에선 구하기 어려운 제품이나 값비싼 제품을 저렴한 서비스가 가능하여 초보자도 조금만 해보면 이용하기 편하고 반품도 편리하답니다 하지만 요즘은 국내 통관불가 품목이 많아 제품별로 잘 확인하고 구매해야 해요 일부 금액 이상 이용 시 한국까지 무료배송 이벤트도 자주 진행 하고 있어요 추천 제품 종합비타민 유산균 식품 임산부 제품 등 홈페이지


TITLE :2449 Similarities:0.8991801578981512
엄마들의 직구 용품 후기 어떤 걸 살까요 엄마들의 직구 용품 국내 온라인 오프라인 여러 곳에서도 쇼핑할 수 있지만 대에 샀어요 국내에서는 가격대가 꽤 나가고 해외 직구제품은 AS에 제한이 있다고 해서 무척고민했어요 근데 어차피 잘 고장 안나고 몇년동안 꾸준하게 썼다는 분이 많길래 as걱정 없이 그냥 직구로 샀어요 다행히 제가 산 곳은 as를 1년간 판매처에서 해준다고 했어요 정말 잘 산 것 같아요 만족해요 아기 장난감 아기 용품 이미지 니나 출산을 준비하는 초보엄마에요 슬슬 출산준비 하면서 아마존에서 직구를 했어요 아기 옷 속싸개 젖병 건조대 기저귀 봉투 그리고 많이들 쓰시는 #타이니러브 수드앤 그루브 모빌도 직구로 구매했어요


TITLE :2448 Similarities:0.8896419294937288
출산용품 면세점 쇼핑리스트 best 10 알뜰하게 구입해요 면세점 출산용품 쇼핑리스트해외여행의 꽃은 면세점 쇼핑 ㅋ 해외로 태교여행을 떠난다면 면세점에 들러 출산용품과 육아용품을 준비해보는건 어떨까요 면세하

---

## Fasttext

In [117]:
fasttext_model = FastText.load('data/tokmom_fasttext.model')
df = pd.read_csv('data/text_for_word2vec.csv')
df = df.dropna()
df = df.reset_index(drop=True)

### Preprocessing for FastText 
- 한글을 자음, 모음 단위로 분해
- 단어 단위로 구분된 ndarray 자료구조로 정제

In [118]:
import re
from soynlp.hangle import decompose, compose

def remove_doublespace(s):
    doublespace_pattern = re.compile('\s+')
    return doublespace_pattern.sub(' ', s).strip()

def encode(s):
    def process(c):
        if c == ' ':
            return c
        jamo = decompose(c)
        # 'a' or 모음 or 자음
        if (jamo is None) or (jamo[0] == ' ') or (jamo[1] == ' '):
            return ' '
        base = jamo[0]+jamo[1]
        if jamo[2] == ' ':
            return base + '-'
        return base + jamo[2]

    s = ''.join(process(c) for c in s)
    return remove_doublespace(s).strip()

#     s = [process(c) for c in s]
#     return s

def decode(s):
    def process(t):
        assert len(t) % 3 == 0
        t_ = t.replace('-', ' ')
        chars = [tuple(t_[3*i:3*(i+1)]) for i in range(len(t_)//3)]
        recovered = [compose(*char) for char in chars]
        recovered = ''.join(recovered)
        return recovered

    return ' '.join(process(t) for t in sdf_features.split())

In [119]:
df['encoded'] = df['text'].apply(encode)

### Vectorizing

In [120]:
def jamo_to_vector(tokens):
    mean_of_vectors = []
    except_tokens = []
    tokens = tokens.split(' ') # 띄어쓰기 단위로 글자 구분을 해준다.
    for token in tokens:
        try:
            vector = fasttext_model.wv.get_vector(token)
            mean_of_vectors.append(vector)
        except KeyError:
            print("{} is not exist in the vocab".format(noun))
            except_tokens.append(token)
            continue
    try:
        mean_of_vectors = sum(mean_of_vectors) / len(mean_of_vectors)
        return mean_of_vectors
    except ZeroDivisionError:
        return np.nan

In [121]:
%%time
df['doc_vector'] = df['encoded'].apply(jamo_to_vector)
df = df.dropna()
df = df.reset_index(drop=True)

CPU times: user 53.7 s, sys: 0 ns, total: 53.7 s
Wall time: 53.7 s


In [122]:
vectors = np.zeros(100,)
for vector in df.doc_vector:
    vectors = np.vstack([vectors, vector])
vectors = vectors[1:, :]
vectors

array([[ 0.86658299, -0.56535929, -0.45894697, ...,  0.00341589,
        -0.33599198,  0.62156469],
       [ 0.53817666, -0.72349644, -0.85632437, ...,  0.05255881,
        -0.50502682,  0.70481634],
       [ 0.50858897, -0.60111189, -0.46155089, ..., -0.24129999,
        -0.67478687,  0.6221779 ],
       ...,
       [ 0.29718897, -0.43003976, -0.75820154, ..., -0.08352479,
        -0.68372095,  0.88945806],
       [ 0.41118783, -0.20348136, -1.08845186, ...,  0.0540762 ,
        -0.61487466,  0.9850949 ],
       [ 0.38487715, -0.41307026, -0.82243931, ..., -0.05015103,
        -0.7017765 ,  0.58764511]])

>예쁘게 만들어야 되는데 이런 식으로 밖에 못만드는게 내 실력 미달인듯... ㅠ

## Normalize extracted feature set
- 유사도 계산의 편의를 위해 0과 1사이의 값으로 정규화

In [123]:
normalizer = Normalizer()
norm_features = normalizer.fit_transform(vectors)
df_features = pd.DataFrame(norm_features,index=df.index.tolist())
df_features.tail()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
19500,-0.157133,0.189910,-0.188528,0.019966,-0.172055,-0.209940,-0.082700,0.022620,-0.019062,0.084801,...,-0.145142,0.096071,0.087845,0.030447,0.101301,0.090200,0.007959,-0.133222,0.048011,-0.011496
19501,0.068911,-0.042133,-0.182900,-0.050632,-0.176763,-0.126192,-0.103488,-0.125923,-0.049064,0.256410,...,-0.097004,0.031894,-0.137346,-0.075640,0.090801,0.080767,-0.024288,0.014092,-0.117959,0.122390
19502,0.040154,-0.058103,-0.102442,0.006907,-0.196867,-0.057263,-0.137394,-0.103822,-0.077189,0.193767,...,-0.177352,0.039965,-0.033948,-0.178392,0.086315,0.064732,0.037422,-0.011285,-0.092378,0.120176
19503,0.052673,-0.026066,-0.139431,0.013634,-0.144553,-0.092317,-0.157348,-0.014650,-0.072456,0.246368,...,-0.127140,0.018352,-0.122910,-0.136783,0.155050,0.061784,0.064725,0.006927,-0.078766,0.126191
19504,0.056155,-0.060268,-0.119996,0.012139,-0.234812,-0.071636,-0.119494,-0.088790,-0.074353,0.295101,...,-0.154728,0.073312,-0.139678,-0.085137,0.125316,0.041369,-0.025539,-0.007317,-0.102391,0.085739


In [124]:
article = df_features.loc[0]
print(article.shape, df_features.shape)
print(np.linalg.norm(article), np.linalg.norm(df_features, axis=1))
similarities=df_features.dot(article) # cosine similarity
top=similarities.nlargest(5)

texts = df.loc[top.index]['text'].tolist()

i = 0
print("<비교할 문서>")
for text in texts:
    df_idx = top.index[i]
    print('TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
    print(text+'\n')
    i = i+1

(100,) (19505, 100)
1.0 [1. 1. 1. ... 1. 1. 1.]
<비교할 문서>
TITLE :0 Similarities:1.0000000000000002
6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요 매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다

TITLE :18912 Similarities:0.9693972701129508
주변에 연

### 가상의 키워드를 입력하여 유사도 계산해보기
- 이번에는 챗봇처럼 짧은 문장(아마 키워드 위주)을 입력받아 유사도 계산을 해보자.
- Fasttext는 word2vec과 달리 단어를 입력받을 경우 에러가 발생하는 문제를 방지할 수 있다!

In [ ]:
while True:
    input_text = input("Input text:")
    if input_text == "exit":
        print("Bye")
        break
        
    tokens = encode(input_text)
    tokens = tokens.split(' ')

    normalizer = Normalizer()
    input_vector = np.array([fasttext_model.wv.get_vector(token) for token in tokens])
    input_vector = sum(input_vector) / len(input_vector)
  
    input_vector = input_vector.reshape(1, -1)
    input_vector = normalizer.fit_transform(input_vector)
    input_vector = input_vector.reshape(-1,)

    similarities=df_features.dot(input_vector)
    top=similarities.nlargest(5)

    texts = df.loc[top.index]['text'].tolist()

    i = 0
    for text in texts:
        df_idx = top.index[i]
        print('\n'+'TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
        print(text+'\n')
        i = i+1
            
#     except:
#         print("'{}'은(는) 제가 아직 잘 이해하지 못하겠어요ㅠㅠ ".format(input_text))
#  6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요 매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다

Input text:중고 육아용품 어디서 구매하는게 안전할까요?

TITLE :2663 Similarities:0.8185125041661083
유모차 선택가이드 개요편 아기와 떠나는 나들이길 필수품 유모차 이것이 진정 BABY ON BOARD 아기와 함께하는 나들이길의 필수품은 현금 카드 댓츠노노 잘 밀리면서도 아기가 편한 유모차랍니다 유모차의 기본 종류와 고르는 팁을 요점정리해 드립니다 유모차 초보운전자들이여 모두 모여 기똥찬 유모차 골라봐요 도대체 유모차가 그냥 유모차 아니었나요 절충형은 뭐고 휴대형은 뭔가요 그리고 신생아때 유모차와 돌지나서 타는 유모차가 달라져야하는 건가요 몇 개를 사야하는거지 자 조급해 마시고 천천히 배워보아요 유모차의 종류 및 사용시기 아하 신생아때는 디럭스 6 7개월부터는 휴대형 그리고 아기요람이 호환되는 절충형 이렇게 세가지로 나누어 지는군요 그럼 각각의 장단점을 한번 알아볼까요 유모차 유형별 장단점과 추천 브랜드먼저 디럭스 유모차의 장단점을 한번 알아볼까요 디럭스 유모차의 경우 몸무게가 가벼운 신생아에게 안정감을 줄 수 있어요 게다가 등받이 조절이 최대로 늘어나기 때문에 유모차에서 잠이 드는 신생아에게는 이보다 더 편할 수 없겠죠 그래서 신생아때는 묵직한 디럭스유모차를 사용하고 아이가 좀 커서 몸무게가 늘어나면 휴대용 유모차로 교체하게 되는 거랍니다 실제로 엄마 아빠가 유모차를 밀어보면 디럭스가 묵직한 느낌이 있어 더 편안해요 하지만 사용기간이 길지 않은데 비해 터치 방식으로 쉽게 접히고 펴지는 기능을 제공하는 유모차가 있음 카시트 호환차에 장착하는 카시트를 유모차에 장착해 사용할 수 있는 기능 대체로 신생아들을 위한 인펀트 카시트를 장착할 수 있는 경우가 많음 개요편 잘 보셨나요 각각의 유모차에 대한 자세한 설명을 보고 싶으시죠 그렇다면 요 정보들도 놓치지 마세요 망설이지 말고 유모차의 모든 것 유모차의 종류 및 구분 절충형 유모차 선택 가이드휴대형 유모차 선택 가이드디럭스형 유모차 선택 가이드 s 리얼한 유모차 후기유모차 

### 자모 단위가 아닌 글자 단위로 했을 때

In [95]:
fasttext_model = FastText.load('data/tokmom_token_fasttext.model')
df = pd.read_csv('data/text_for_word2vec.csv')
df = df.dropna()
df = df.reset_index(drop=True)

### Preprocessing for FastText 
- 한글을 글자 단위로 분해
- 단어 단위로 구분된 ndarray 자료구조로 정제

In [96]:
df['encoded'] = df['text'].apply(tokenize)

In [97]:
df

,text,encoded
0,6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 ...,"[6, 살, 남, 아, 입, 니, 다, 회사, 복직, 후, 3, 살, 부터, 주, ..."
1,요즘 들어 아빠가 아이에게 다가가려고 하면 엄마한테 갈 거야 하면서 엄마를 유독 찾...,"[요즘, 들, 어, 아빠, 가, 아이, 에게, 다가가, 려고, 하, 면, 엄마, 한..."
2,둘째가 태어나면 첫째가 힘들 거라는 건 예상하고 있었지만 툭하면 삐치고 울고 좀처럼...,"[둘째, 가, 태어나, 면, 첫째, 가, 힘들, 거, 라는, 건, 예상, 하, 고,..."
3,20개월 남아가 있고 임신 8개월차로 두 아기의 엄마입니다 첫 아이는 잠자리 독립을...,"[20, 개월, 남아, 가, 있, 고, 임신, 8, 개월, 차, 로, 두, 아기, ..."
4,14개월 남자 아기인데 심하게 깨무는 버릇이 있어요 전에는 졸릴 때 주로 물곤 했는...,"[14, 개월, 남자, 아기, 인데, 심하, 게, 깨무, 는, 버릇, 이, 있, 어..."
...,...,...
19500,토끼띠의 단점,"[토끼띠, 의, 단점]"
19501,국가영어능력평가 준비 올해부터 본격적으로 시행되는 국가영어능력평가에 대해서 소개해드...,"[국가, 영어, 능력, 평가, 준비, 올해, 부터, 본격, 적, 으로, 시행, 되,..."
19502,워킹맘이든 전업맘이든 아이들에게 관심과 사랑을 쏟아야 하는 것은 분명합니다 평소 아...,"[워킹, 맘, 이, 든, 전업, 맘, 이, 든, 아이, 들, 에게, 관심, 과, 사..."
19503,검사결과 지민이의 강점지능은 언어지능과 음악지능였습니다 엄마는 지민이가 집에서나 학...,"[검사, 결과, 지민, 이, 의, 강점, 지능, 은, 언어, 지능, 과, 음악, 지..."


### Vectorizing

In [98]:
def make_to_vector(tokens):
    mean_of_vectors = []
    except_tokens = []
    for token in tokens:
        try:
            vector = fasttext_model.wv.get_vector(token)
            mean_of_vectors.append(vector)
        except KeyError:
#             print("{} is not exist in the vocab".format(noun))
            except_tokens.append(token)
            continue
    try:
        mean_of_vectors = sum(mean_of_vectors) / len(mean_of_vectors)
        return mean_of_vectors
    except ZeroDivisionError:
        return np.nan

In [99]:
%%time
df['doc_vector'] = df['encoded'].apply(make_to_vector)
df = df.dropna()
df = df.reset_index(drop=True)

CPU times: user 18.8 s, sys: 0 ns, total: 18.8 s
Wall time: 18.8 s


In [100]:
vectors = np.zeros(100,)
for vector in df.doc_vector:
    vectors = np.vstack([vectors, vector])
vectors = vectors[1:, :]
vectors

array([[ 0.24673061, -0.27795058,  0.24404778, ...,  1.11242867,
         1.08872926,  0.48421091],
       [ 0.01572265, -0.09072687,  0.1175985 , ...,  0.94063079,
         1.27431548,  0.4101868 ],
       [ 0.12905237,  0.04540736,  0.11949921, ...,  1.09621668,
         1.10972857,  0.29464814],
       ...,
       [-0.20894928, -0.15436162,  0.18836324, ...,  1.18231559,
         1.03075528,  0.16409379],
       [ 0.04918776, -0.01805375,  0.07689283, ...,  1.29379928,
         1.25442481,  0.10543385],
       [-0.05928177, -0.18643324,  0.17147981, ...,  1.32759964,
         1.09847867,  0.12419592]])

>예쁘게 만들어야 되는데 이런 식으로 밖에 못만드는게 내 실력 미달인듯... ㅠ

## Normalize extracted feature set
- 유사도 계산의 편의를 위해 0과 1사이의 값으로 정규화

In [101]:
normalizer = Normalizer()
norm_features = normalizer.fit_transform(vectors)
df_features = pd.DataFrame(norm_features,index=df.index.tolist())
df_features.tail()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
19500,-0.082703,-0.107066,0.020534,-0.013594,0.011498,0.011114,-0.008895,0.004951,-0.031599,0.098213,...,0.229741,-0.229231,-0.021993,0.069794,0.057802,0.117444,-0.019022,0.094695,-0.264936,0.100297
19501,-0.013324,-0.043249,0.056210,0.023133,-0.051027,0.018710,-0.152465,-0.099115,-0.187265,0.053776,...,0.152398,-0.009613,0.017098,-0.036609,0.090501,0.225238,-0.139409,0.236008,0.175327,0.044390
19502,-0.035021,-0.025872,0.031570,0.034131,-0.015111,-0.030294,-0.204873,-0.154331,-0.142759,-0.012280,...,0.220279,-0.101568,0.006249,-0.106006,0.069603,0.127194,-0.097784,0.198160,0.172758,0.027503
19503,0.007437,-0.002730,0.011626,0.006192,-0.047806,-0.003482,-0.192131,-0.096706,-0.142590,0.030369,...,0.262496,-0.007289,0.031681,-0.009989,0.023988,0.201436,-0.076635,0.195617,0.189663,0.015941
19504,-0.009856,-0.030996,0.028509,0.039489,-0.057810,0.005713,-0.177771,-0.106272,-0.183521,0.011650,...,0.219025,0.001136,-0.027511,-0.026783,0.049193,0.167402,-0.071403,0.220721,0.182628,0.020648


In [102]:
article = df_features.loc[0]
print(article.shape, df_features.shape)
print(np.linalg.norm(article), np.linalg.norm(df_features, axis=1))
similarities=df_features.dot(article) # cosine similarity
top=similarities.nlargest(5)

texts = df.loc[top.index]['text'].tolist()

i = 0
print("<비교할 문서>")
for text in texts:
    df_idx = top.index[i]
    print('TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
    print(text+'\n')
    i = i+1

(100,) (19505, 100)
1.0 [1. 1. 1. ... 1. 1. 1.]
<비교할 문서>
TITLE :0 Similarities:1.0000000000000002
6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요 매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다

TITLE :2143 Similarities:0.972685979454132
워킹맘의 고민

### 가상의 키워드를 입력하여 유사도 계산해보기
- 이번에는 챗봇처럼 짧은 문장(아마 키워드 위주)을 입력받아 유사도 계산을 해보자.
- Fasttext는 word2vec과 달리 단어를 입력받을 경우 에러가 발생하는 문제를 방지할 수 있다!

In [104]:
while True:
    input_text = input("Input text:")
    if input_text == "exit":
        print("Bye")
        break
        
    tokens = tokenize(input_text)
#     tokens = tokens.split(' ')

    input_vector = np.array([fasttext_model.wv.get_vector(token) for token in tokens])
    input_vector = sum(input_vector) / len(input_vector)
  
    input_vector = input_vector.reshape(1, -1)
    input_vector = normalizer.fit_transform(input_vector)
    input_vector = input_vector.reshape(-1,)

    similarities=df_features.dot(input_vector)
    top=similarities.nlargest(5)

    texts = df.loc[top.index]['text'].tolist()

    i = 0
    for text in texts:
        df_idx = top.index[i]
        print('\n'+'TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
        print(text+'\n')
        i = i+1
            
#     except:
#         print("'{}'은(는) 제가 아직 잘 이해하지 못하겠어요ㅠㅠ ".format(input_text))
#  6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요 매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다

Input text:독박육아 개짜즡ㅇ나요

TITLE :18099 Similarities:0.5522195472923174
직장 인근에서도 참여 가능하도록 자치구까지 확대4월초 10개 자치구 선정


TITLE :18169 Similarities:0.5371410335864484
12일 서울코엑스에서 개막한 제27회 베이비페어는 개막 첫날부터 관람객이 밀려 대성황을 이루었다 국내 최대 규모로 초보부모와 임산부에게 임신 출산 육아정보를 제공해 주는 베페베이비페어는 지난 26회 기준 누적 관람객 220만 명을 돌파했다 이번 페어에는 국내외 임신 출산 육아 교육과 관련 국내외 140개사 950여 부스 350여 개 브랜드가 대거 참가했다베페 베이비페어를 사진으로 정리했다


TITLE :9040 Similarities:0.5358133748405807
내년 공립유치원 교사 선발 인원이 1018명으로 확정됐다 지난 6월에 발표한 선발예상 인원보다 2배 늘어났다 교육부와 전국 시도교육청은 2019학년도 공립유치원 교사 임용후보자 선정경쟁시험 시행계획을 13일 각 시도교육청 확정·발표했다 오는 11월 10일에 각 시도교육청별로 치러질 2019학년도 공립유치원교사 임용시험 신규선발 규모는 1018명으로 지난 6월에 발표했던 선발예상 인원499명보다 2배 늘어난 수준이다 시도별로 보면 경기 250명 서울 170명 경남 150명 전남 56명 부산 43명 충북 43명 전북 40명 대구 37명 대전 36명 강원 33명 경북 31명 세종 30명 충남 30명 제주 23명 인천 21명 광주 14명을 선발한다


TITLE :4494 Similarities:0.5354791606158337
홈메이드 탄산수 제조기 소다스트림은 오는 18일부터 31일까지 이마트 80개점일렉트로마트 전점 일반점포 41개점에서 소다스트림 썸머 이마트팩 한정상품 특가행사를 실시한다고 17일 밝혔다 소다스트림은 이마트 80개점에서 이마트 단독 썸머 이마트팩을 최대 4만 원할인 금액으로 소비자들에게 선보인다 2가지 

Input text:유튜브

TITLE :9066 Similarities:0.7435954143850535
주부들이 선호하는 영유아 브랜드를 선정하는 2018 영유아 브랜드 선호도 조사의 2차 결선이 시작됐다 육아전문지 베이비뉴스는 오는 30일까지 베이비뉴스 예비맘 및 육아맘들을 대상으로 한 2018 영유아 브랜드 선호도 조사 2차 결선을 실시한다 2018 영유아 브랜드 선호도 조사 2차 결선에서는 예선을 거쳐 올라온 임신 출산 육아 교육 여성 주부 생활용품 등 총 40개 분야의 5개 브랜드가 경쟁한다 1위 브랜드는 온라인 객관식 설문을 통해 선정된다 설문 참여자들에겐 추첨을 통해 유모차2명 유아 트램펄린3명 유아 블록5명 유아 승용완구5명 유아 세제세트25명 커피 기프티콘100명 등을 증정한다 결선 결과는 다음달 5일 베이비뉴스 홈페이지 및 공식 포스트에서 확인할 수 있다 최규삼 베이비뉴스 대표는 베이비뉴스는 올바른 소비의 기준을 마련하고자 2013년부터 매년 온라인과 오프라인으로 예비맘과 육아맘들을 대상으로 한 영유아 여성 및 주부 브랜드 선호도조사를 시행해오고 있다며 브랜드 선호도 조사로 육아 용품 구매와 여성 및 주부 관련 제품 소비에 도움이 되길 바란다고 전했다 2018 영유아 브랜드 선호도 조사 2차 결선에 오른 분야별 브랜드가나다 순는 아래와 같다 가구리바트 에몬스 이케아 일룸 한샘가전건조기SK매직 삼성 LG 위니아 위닉스가전공기청정기삼성 샤오미 LG 위닉스 코웨이가전세탁기대우 삼성 SK매직 LG 위닉스가전청소기다이슨 삼성 샤오미 LG 일렉트로룩스기저귀군 마미포코 킨도 팸퍼스 하기스대형마트농협 하나로마트 롯데마트 이마트 코스트코 홈플러스물티슈물따로 베베숲 앙블랑 페넬로페 하기스분유남양 매일 압타밀 일동 파스퇴르비타민고려은단 비타민C100 뉴트리라이트 더블X 닥터써디니드롭스 센트롬 솔가 멀티비타민상조교원라이프 보람상조 예다함 조은라이프 프리드라이프상처치료제듀오덤 마데카솔 메디폼 비판텐 후시딘세제BB 블랑101 비트 에티튜드 퍼실쇼핑몰11번가 위메프 

Input text:집에 가고 싶다

TITLE :6818 Similarities:0.7565207906833968
엄마 이러다 늦겠어요 빨리 빨리 아이는 현관문 앞에 서서 큰소리로 외쳤다 이미 신발신고 가방도 메고 대기 중이다 알았어 엄마 다 했어 잠깐만 엄마 빨리 가요 빨리요 엄마도 널 빨리 보내고 싶거든 발을 동동 구르며 빨리 빨리를 외치는 아이의 재촉하는 소리가 귀찮긴 한데 나쁘지 않다 언제부터 재촉했다고 저러지 싶어 웃음이 나온다 아이는 지금 빨리 어린이집에 가자고 노래를 부르는 중이다 아이는 작년부터 어린이집에 다니기 시작했다 그때와 지금이 이렇게 다를 수 있나 싶은 요즘이다 그땐 단체생활이 처음인 아이도 아이를 처음 사회로 보내는 엄마도 모두 불안했다 잘할 수 있을까 잘 지낼 수 있을까 가방 메고 어린이집까지 걸어가서 띵동 벨 누르면 선생님이 나오실 거야 안녕하세요 인사하고 엄마 빠빠이 할거야 친구들이랑 신나게 놀고 점심 먹자 코자하고 나면 엄마가 데리러 갈게 한 달을 준비했다 아이가 엄마와 헤어지는 상황을 미리 말해주고 직접 시뮬레이션까지 하며 웃으며 빠빠이 하는 연습을 꾸준히 했다 연습과 실전은 달랐다 아이는 어린이집에 간 첫날 대성통곡 하듯 울었다 멀어지는 엄마를 바라보며 이게 아닌데 하는 눈치였다 엉엉 울며 엄마에게 손을 뻗는 아이를 뒤로하고 집으로 돌아왔다 단호하게 뒤돌아섰지만 집에서 기다리는 내 속도 말이 아니었다 일주일을 꼬박 울던 아이는 더 이상 울지 않았다 한결이가 정말 적응력이 뛰어나네요 선생님의 말씀에 역시 우리 아들하며 목에 힘이 들어갔다 울지 않고 가면 성공이라고 우리 아이가 이렇게 잘 적응할 줄 몰랐다며 좋아했다 점심도 먹고 낮잠도 자고 오기로 했다 오예 하지만 아이가 잘 적응했다고 생각한 건 착각이었을까 아이는 울지만 않을 뿐 어린이집에 가는 걸 정말 싫어했다 어린이집 재미없어 가기 싫어 매일 아침 아이는 옷도 안 입겠다 신발도 안 신겠다며 나를 피해 도망 다녔다 억지로 끌고 나가서 엘리베이터를 타는 순간까지도 어린이집 

Input text:아기 장난감 직접 사용해본 후기

TITLE :1609 Similarities:0.7651979449192519
해외직구사이트 육아용품 여기서 마련하세요 이미지 출처 아이아트 육아는 글로벌 장비빨 해외직구사이트 육아용품 여기서 마련하세요 안 해본 사람은 있어도 한 번만 해본 사람은 없다는 육아용품 해외직구 임신 출산 용품 부터 아기 옷 기저귀 장난감까지 국내에선 구하기 어려운 제품이나 값비싼 제품을 저렴한 서비스가 가능하여 초보자도 조금만 해보면 이용하기 편하고 반품도 편리하답니다 하지만 요즘은 국내 통관불가 품목이 많아 제품별로 잘 확인하고 구매해야 해요 일부 금액 이상 이용 시 한국까지 무료배송 이벤트도 자주 진행 하고 있어요 추천 제품 종합비타민 유산균 식품 임산부 제품 등 홈페이지


TITLE :2297 Similarities:0.736677701826393
아기침대 세탁 및 소독 어떻게 하나요 이미지 출처 아이 아트 청결한 관리가 생명 아기침대 세탁 및 소독 어떻게 하나요 우리 아이가 하루 중 반 이상을 머무는 아이의 잠자리 청결은 매우 중요하죠 면역력이 약한 아이들을 위해 침대 세탁은 꼼꼼하게 해야 해요 아기침대 세탁 및 소독 방법과 정보를 알려드릴게요 1 직접 세탁법 아기 침대 중 범퍼 침대나 이동식 침대 등은 집에서 세탁할 수 있는 제품들이 많아요 힘은 들지만 집에서 엄마아빠의 손으로 꼼꼼하게 청소할 수 있어요 범퍼침대 세탁 방법 이미지 출처 아이조아 페어리 1 겉커버와 속커버를 분리해 주세요 2 겉커버는 뒤집어 세탁망에 넣고 울코스 란제리코스 에서 중성세제로 세탁해주세요 3 속커버는 대부분 방수처리 되어 있어요 마른천에 베이킹소다수 혹은 식초물을 묻혀 닦은 뒤 햇볕에서 완벽하게 말려주세요 4 세탁한 겉커버도 햇볕에서 완벽하게 말려주세요 이동식 아기침대 세탁 방법 이미지 출처 뉴나 1 접이식이나 플레이야드형 등의 이동식 아기침대는 분해되는 형태의 경우는 먼저 분리를 해주세요 분리가 안되는 제품은 그대로 욕실로 가져가세요 2 

KeyboardInterrupt: 